In [2]:
import pandas as pd
import gzip
import sys
from Bio import SeqIO

In [4]:
file_names = pd.read_csv("init/promoters.tsv", sep="\t", header=0)

# Test just the first 100 lines un the GFF3 files
for index, row in file_names.iterrows():
    # Should add a line to delete a file if already exists
    genotype = row[0]
    source   = row[1]
    fasta    = row[2]
    gff3     = row[3]
    size     = row[4]
    output   = row[5]
    print(genotype, fasta, gff3, size, output)
    promoter_parser(genotype, fasta, gff3, size, output)

Arabidopsis Athaliana_167_TAIR10.fa.gz Athaliana_167_TAIR10.gene.gff3.gz 1000 Athaliana_167
Maize Zmays_493_APGv4.fa.gz Zmays_493_RefGen_V4.gene.gff3.gz 1000 Zmays_493_APGv4


C:\Users\SagerFish\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3347: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Sorghum Sbicolor_454_v3.0.1.fa.gz Sbicolor_454_v3.1.1.gene.gff3.gz 1000 Sbicolor_454_v3.0.1


In [3]:
# Read the GFF file that contains the gene IDs in the 8th column
# Find the transcription start site (ATG) for each gene.
# Based on phytozome GFF3 files. Adjustments to be made for other sources
# Assumes row are ordered: gene > mRNA > CDS
# Assumes CDS are ordered for both fwd and rev genes and 1st CDS is ATG

# This works but I am going to load the fasta as a dictionary for preformance
def promoter_extractor(genotype, gene_id, seq, start, strand, size, kind, output):
    inbred_name = genotype
    
    if strand == "+":
        seq = seq[start-size:start+2]
    if strand == "-":
        # I need to reverse complement this
        seq = seq[start-3:start+size].reverse_complement()
    
    # test that the last 3 base-pairs are the correctly the start codon
    #if seq[-3:] != "ATG" and seq[-3:] != "atg":
        #print("Start codon not present at the end of the promoter:", gene_id, seq[-3:])

    # remove the start-codon
    seq = seq[:-3]
                
    
    with gzip.open("inputs/promoters/"+output+"_1kb_"+kind+".fasta.gz", 'a') as output:
        # A bytes-like object is required, not str
        output.write(('>'+gene_id+'\n').encode())
        output.write((str(seq)+'\n').encode())


def promoter_parser(genotype, fasta, gff, size, output):
    
    with gzip.open("inputs/genomes/"+fasta, "rt") as handle:
        # rec.id can take different formats (Chr1, chr1...) and if just 1 convert to str
        seq_dict = {str(rec.id) : rec.seq for rec in SeqIO.parse(handle, "fasta")}
    
    gff = pd.read_csv("inputs/gff3/"+gff, sep="\t", header=None, compression='gzip', comment='#')
    gff = gff[gff[2].isin(["gene", "mRNA", "CDS"])]

    gene_id = None
    longest = None
    start   = None
    skip_row= True

    for index, row in gff.iterrows():
    #for index, row in gff.iterrows():
        gff3_name = row[8]

        if row[2] == "gene":
            # Every gene should have a longest, if not print message
            if skip_row == False:
                print("No longest for gene:", gene_id)
            
            gene_id = gff3_name[gff3_name.find("Name=")+5:].split(";")[0]

            # Extract 1kb promoter starting from the ATG of the annotation gene
            kind = "TSS"
            
            # Get the start coordinates of the gene depending on orientation
            if row[6]=="+":
                start = row[3]
            else:
                start = [4]
            
            promoter_extractor(genotype, gene_id, seq_dict[str(row[0])], row[3], row[6], size, kind, output)

            skip_row = False

        if row[2] == "mRNA":
            longest = int(gff3_name[gff3_name.find("longest")+8:gff3_name.find("longest")+9])

        if skip_row:
            continue

        if row[2] == "CDS" and longest == 1 and row[6]=="+":
            start = row[3]
            skip_row = True
            kind = "ATG"
            promoter_extractor(genotype, gene_id, seq_dict[str(row[0])], start, row[6], size, kind, output)
            #print(gene_id, longest, start, row[6])

        if row[2] == "CDS" and longest == 1 and row[6]=="-":
            start = row[4]
            skip_row = True
            kind = "ATG"
            promoter_extractor(genotype, gene_id, seq_dict[str(row[0])], start, row[6], size, kind, output)
            #print(gene_id, longest, start, row[6])  